# Explorador de Datos y Detección de Anomalías en Series de Tiempo

Este notebook demuestra el uso de la clase `TimeSeriesAnomalyDetector` para analizar series de tiempo y detectar anomalías utilizando diferentes métodos.

## Contenido:
1. Carga y preparación de datos
2. Inicialización del detector de anomalías
3. Aplicación de métodos de detección
4. Visualización de resultados

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from anomaly_detection import TimeSeriesAnomalyDetector
import warnings
warnings.filterwarnings('ignore')

## 1. Carga y Exploración de Datos

Cargamos la serie de tiempo desde el archivo CSV y realizamos una exploración inicial.

In [ ]:
# Cargar datos del CSV
df = pd.read_csv('csv/time-series.csv')
print("Primeras filas del dataset:")
df.head()

In [ ]:
# Información general del dataset
print("\nInformación del dataset:")
df.info()

print("\nEstadísticas descriptivas:")
df.describe()

In [ ]:
# Preparar el DataFrame para el análisis
# Convertir Timestamp a datetime y establecer como índice
df_prepared = df.copy()
df_prepared['Timestamp'] = pd.to_datetime(df_prepared['Timestamp'])
df_prepared.set_index('Timestamp', inplace=True)
df_prepared['Value'] = pd.to_numeric(df_prepared['Value'], errors='coerce')

print("Datos preparados (la limpieza se hace automáticamente en add_series):")
df_prepared.head()

## 2. Inicialización del Detector de Anomalías

Creamos una instancia de `TimeSeriesAnomalyDetector` y agregamos nuestra serie de tiempo.

In [ ]:
# Inicializar el detector
detector = TimeSeriesAnomalyDetector()

# Agregar la serie de tiempo (la limpieza se hace automáticamente)
detector.add_series('SeriePrincipal', df_prepared)

print("Serie agregada exitosamente.")
print(f"Series disponibles: {list(detector.dataframes.keys())}")
print(f"Forma de los datos limpios: {detector.dataframes['SeriePrincipal'].shape}")

## 3. Aplicación de Métodos de Detección de Anomalías

Aplicamos el método Isolation Forest para detectar anomalías en la serie de tiempo.

In [ ]:
# Aplicar Isolation Forest
detector.apply_isolation_forest(
    series_name='SeriePrincipal',
    target_col='Value',
    n_estimators=100,
    contamination=0.01  # Esperamos 1% de anomalías
)

print("Isolation Forest aplicado exitosamente.")

In [ ]:
# Verificar resultados
results = detector.results['SeriePrincipal']
print("Resultados de la detección:")
print(results.head())

# Contar anomalías detectadas
anomalies_count = (results['is_anomaly_IF'] == -1).sum()
total_points = len(results)
print(f"\nAnomalías detectadas: {anomalies_count} de {total_points} puntos ({anomalies_count/total_points*100:.2f}%)")

## 4. Visualización de Resultados

Creamos una visualización interactiva que muestra la serie de tiempo y las anomalías detectadas.

In [ ]:
# Crear visualización
fig = detector.plot_anomalies(
    series_name='SeriePrincipal',
    target_col='Value',
    methods_to_plot=['IF']  # IF = Isolation Forest
)

# Mostrar el gráfico
fig.show()

## 5. Análisis Adicional

Exploramos más detalles sobre las anomalías detectadas.

In [ ]:
# Obtener las anomalías detectadas
anomalies = results[results['is_anomaly_IF'] == -1]
print("Anomalías detectadas (primeras 10):")
print(anomalies.head(10))

# Estadísticas de las anomalías
if len(anomalies) > 0:
    print("\nEstadísticas de las anomalías:")
    anomaly_values = df_prepared.loc[anomalies.index, 'Value']
    print(anomaly_values.describe())
    
    # Comparar con valores normales
    normal_values = df_prepared.loc[results[results['is_anomaly_IF'] == 1].index, 'Value']
    print("\nEstadísticas de valores normales:")
    print(normal_values.describe())

## 6. Extensión a Múltiples Series (Ejemplo)

Demostramos cómo agregar múltiples series de tiempo al detector.

In [ ]:
# Crear una segunda serie con algunos valores atípicos para demostración
df2 = df_prepared.copy()
df2['Value'] = df2['Value'] * 3 # Modificar ligeramente

# Agregar algunos valores atípicos
outlier_indices = df2.sample(frac=0.02).index  # 2% de outliers
df2.loc[outlier_indices, 'Value'] = df2.loc[outlier_indices, 'Value'] * 4

# Agregar la segunda serie
detector.add_series('SerieModificada', df2)

print("Segunda serie agregada.")
print(f"Series disponibles: {list(detector.dataframes.keys())}")

In [ ]:
# Aplicar análisis a la segunda serie
detector.apply_isolation_forest(
    series_name='SerieModificada',
    target_col='Value',
    contamination=0.02  # Más contaminación esperada
)

# Visualizar comparación
fig2 = detector.plot_anomalies(
    series_name='SerieModificada',
    target_col='Value',
    methods_to_plot=['IF']
)

fig2.show()

## Conclusiones

Este notebook ha demostrado:

1. **Carga y preparación** de datos de series de tiempo
2. **Uso de la clase TimeSeriesAnomalyDetector** para análisis flexible
3. **Aplicación de Isolation Forest** para detección de anomalías
4. **Visualización interactiva** con Plotly
5. **Extensibilidad** para múltiples series y métodos futuros

La clase está diseñada para ser fácilmente extensible con nuevos métodos de detección de anomalías como One-Class SVM, Hampel Filter, etc.